Category 	Available 	Requirements
Total Points 	100	
Create Models	50	Create a logistic regression model and a support vector machine model for the classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable timeframe. 
Model Advantages	10	Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.
Interpret Feature Importance	30	Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?
Interpret Support Vectors	10	Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try subsampling your data to train the SVC model— then analyze the support vectors from the subsampled dataset.

1. You must include all of your data wrangling code, if you want to receive full credit.  Otherwise, your work is not reproducible.  This is true for all submissions. You are the only group who left this out. I need to see what you started with and all transformations. The dataset must still meet the minimum size after all transformations.  I will not accept future labs without this code included.  

2. I have stated multiple times that the minimum dataset size is 15 columns x 30,000 rows = 450,000 elements. My feedback clearly states that what you submitted is currently at 16 * 9731 = 155,696 elements. As I said in the feedback: "In future labs, you will need to increase the number of fields you use for modeling to achieve at least 15 * 30,000 = 450,000 elements.". 

3. Unfortunately for Lab 1, it really makes no difference what your yearly tables may look like at this point, because they were not included in your submission? 

4. I see above that the dataset you produced for each year has many more fields / columns / features that you could have included to meet the minimum size requirements?  The starting point should have likely been that entire dataset.  I am also slightly concerned about your statement that "merging is a simple inner join activity" for a number of reasons. The dataset you started with does not appear to be joined at all.  There is a year column in the dataset.  This gives me the impression that you actually unioned the data by year (which is what I would have expected). However, I really have no clue what you did, because you chose not to share that with me.

Thoughts on inner joining this data:
A number of public school campuses open and close each school year.  If you are joining by unit_code / agency_code, your analysis will only include school campuses open during the entire duration of the data.
Any classification model you create in a "joined" dataset will likely have duplicated columns for each year.  This means that the feature importances for the model will be dominated by prior years.  For example, if you build a model to predict graduation rates, the model will use all prior year's graduation rates for the prediction.  It will also be very accurate.  However, I am not sure what benefit such a model would provide to the public school system?
Typically, you would not include any graduation rate features in your graduation rate model training data at all. (we can talk about this more, if needed).      

# MiniLab SVM and Logistic Regression
#### 06/14/2020
#### Yang Zhang, Reannan McDaniel, Jonathon Roach, Fred Poon

## Data Preparation
### One-Hot Encoding

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For multiple line outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

schoolData = pd.read_csv('C:/Users/taniat470s/Desktop/SMU_course/DS7331/PublicSchools2014to2017_YZ.csv') # read in the csv file

In [32]:
#Locate the categorical variables in dataset
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
uniqueThreshold = 25

#Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

#Review dataset contents one hot high unique value drops
print('*********After: Removing columns with >= uniqueThreshold unique values***********')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(oneHotUniqueValueCols))

*********After: Removing columns with >= uniqueThreshold unique values***********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9731 entries, 0 to 9730
Columns: 250 entries, szip_ad to Year
dtypes: float64(237), int64(2), object(11)
memory usage: 18.6+ MB

Columns Deleted:  9


In [33]:
#Keep an original copy of the Data
schoolData_org = schoolData

#Pop out the SPG information from the Data
schoolData_SPG = schoolData.pop('SPG Grade')
schoolData_SPGScore = schoolData.pop('SPG Score')

In [20]:
schoolData.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9731 entries, 0 to 9730
Data columns (total 248 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   szip_ad                              float64
 1   category_cd                          object 
 2   calendar_type_txt                    object 
 3   sna_pgm_type_cd                      object 
 4   school_type_txt                      object 
 5   calendar_only_txt                    object 
 6   title1_type_cd                       object 
 7   esea_status                          object 
 8   student_num                          float64
 9   lea_avg_student_num                  float64
 10  st_avg_student_num                   float64
 11  Grad_project_status                  object 
 12  Math I_Size                          float64
 13  lea_total_expense_num                float64
 14  lea_salary_expense_pct               float64
 15  lea_services_expense_pct             

In [34]:
# Remove All the Columns that directly used to calculate SPG Grade and Score
schoolData.drop(schoolData.iloc[:, 22:162], inplace=True, axis=1)
schoolData.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9731 entries, 0 to 9730
Data columns (total 108 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   szip_ad                              float64
 1   category_cd                          object 
 2   calendar_type_txt                    object 
 3   sna_pgm_type_cd                      object 
 4   school_type_txt                      object 
 5   calendar_only_txt                    object 
 6   title1_type_cd                       object 
 7   esea_status                          object 
 8   student_num                          float64
 9   lea_avg_student_num                  float64
 10  st_avg_student_num                   float64
 11  Grad_project_status                  object 
 12  Math I_Size                          float64
 13  lea_total_expense_num                float64
 14  lea_salary_expense_pct               float64
 15  lea_services_expense_pct             

In [35]:
#Isolate remaining categorical variables
begColumnCt = len(schoolData.columns)

#one hot encode categorical variables
schoolData = pd.get_dummies(data=schoolData,drop_first=True)

In [36]:
#Determine change in column count
endColumnCt = len(schoolData.columns)
columnsAdded = endColumnCt - begColumnCt


#Review dataset contents one hot high unique value drops
print('Columns To One-Hot Encode: ', len(sD_nominal.columns))
print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
schoolData.info(verbose=False)
print ('\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded)

Columns To One-Hot Encode:  20

*********After: Adding New Columns Via One-Hot Encoding*************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9731 entries, 0 to 9730
Columns: 148 entries, szip_ad to Category_Cd_T
dtypes: float64(96), int64(2), uint8(50)
memory usage: 7.7 MB

New Columns Created Via One-Hot Encoding:  40


In [37]:
# Dimension of Dataset
schoolData.shape

schoolData.head()

(9731, 148)

,szip_ad,student_num,lea_avg_student_num,st_avg_student_num,Math I_Size,lea_total_expense_num,lea_salary_expense_pct,lea_services_expense_pct,lea_supplies_expense_pct,lea_instruct_equip_exp_pct,...,Grad_project_status_Y,EVAAS Growth Status_Exceeded,EVAAS Growth Status_Met,EVAAS Growth Status_NotMet,Category_Cd_A,Category_Cd_E,Category_Cd_H,Category_Cd_I,Category_Cd_M,Category_Cd_T
0,27253.0,78.0,954.0,837.0,0.0,8028.59,0.613,0.078,0.086,0.011,...,0,0,0,0,0,0,1,0,0,0
1,27253.0,539.0,518.0,496.0,0.0,8028.59,0.613,0.078,0.086,0.011,...,0,1,0,0,0,1,0,0,0,0
2,27244.0,547.0,518.0,496.0,0.0,8028.59,0.613,0.078,0.086,0.011,...,0,0,1,0,0,1,0,0,0,0
3,27217.0,800.0,768.0,665.0,26.0,8028.59,0.613,0.078,0.086,0.011,...,0,0,0,1,0,0,0,0,1,0
4,27215.0,664.0,518.0,496.0,0.0,8028.59,0.613,0.078,0.086,0.011,...,0,1,0,0,0,1,0,0,0,0


### Feature Selection

We original have 148 features in the data table. We don't want to include this many features for model feasibility, so we conduct a feature selection process to choose the most important features to use.
There are many ways to conduct feature selection. In this project we use random forest and plot out feature importances of all features. The work below is not for feasible model construction (we didn't split train/test) but just for picking out the most important variables to use.  

In [38]:
# Random Forest apply to X with all variables
from sklearn.ensemble import RandomForestClassifier
clfa=RandomForestClassifier(n_estimators=100)
clfa.fit(schoolData,schoolData_SPG)
y_pred=clfa.predict(schoolData)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Accuracy = 1.0 does not surprise us because we didn't to train/test splitting. 

In [39]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(schoolData_SPG, y_pred))

Accuracy: 1.0


Below plot reflect what features are important and their according weights.

In [40]:
import eli5
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(clfa, random_state=1).fit(schoolData,schoolData_SPG)
eli5.show_weights(perm, feature_names = schoolData.columns.tolist(), top=50)

Weight,Feature
0.0139 ± 0.0021,EVAAS Growth Status_NotMet
0.0137 ± 0.0024,TotalTargets_pTarget_PctMet
0.0059 ± 0.0007,MinorityMalePct
0.0028 ± 0.0008,title1_type_cd_Y
0.0020 ± 0.0006,MinorityFemalePct
0.0012 ± 0.0002,short_susp_per_c_num
0.0007 ± 0.0002,EVAAS Growth Status_Exceeded
0.0007 ± 0.0005,BlackMalePct
0.0005 ± 0.0003,AsianFemalePct
0.0003 ± 0.0003,avg_daily_attend_pct


For our purpose we just pick the first 50 variables to use. Notice that in the below variable list there are both continuous variables and the categorical variables after one-hot encoding. 

In [41]:
schoolData_pick = schoolData[[
'TotalTargets_pTarget_PctMet',
'EVAAS Growth Status_NotMet',
'MinorityMalePct',
'MinorityFemalePct',
'title1_type_cd_Y',
'avg_daily_attend_pct',
'short_susp_per_c_num',
'BlackMalePct',
'AsianFemalePct',
'student_num',
'HispanicMalePct',
'SciGr5&8_pTarget_PctMet',
'tchyrs_0thru3_pct',
'tchyrs_11plus_pct',
'Accomplished_TCHR_Standard 2_Pct',
'Accomplished_TCHR_Standard 1_Pct',
'Developing_TCHR_Standard 1_Pct',
'Developing_TCHR_Standard 2_Pct',
'Developing_TCHR_Standard 3_Pct',
'Accomplished_TCHR_Standard 4_Pct',
'4-10 Years_LEA_Exp_Pct_Prin',
'Developing_TCHR_Standard 4_Pct',
'Developing_TCHR_Standard 5_Pct',
'10+ Years_LEA_Exp_Pct_Prin',
'Accomplished_TCHR_Standard 3_Pct',
'Accomplished_TCHR_Standard 5_Pct',
'lea_state_perpupil_num',
'st_emer_prov_teach_pct',
'pct_GCE_ALL',
'MathGr3-8_pTarget_PctMet',
'lea_sat_avg_score_num',
'lea_federal_perpupil_num',
'lea_local_perpupil_num',
'nbpts_num',
'Distinguished_TCHR_Standard 2_Pct',
'_1yr_tchr_trnovr_pct',
'lateral_teach_pct',
'0-3 Years_LEA_Exp_Pct_Prin',
'lea_flicensed_teach_pct',
'lea_tchyrs_4thru10_pct',
'lea_tchyrs_11plus_pct',
'lea_nbpts_num',
'lea_advance_dgr_pct',
'lea_1yr_tchr_trnovr_pct',
'lea_emer_prov_teach_pct',
'st_flicensed_teach_pct',
'st_tchyrs_0thru3_pct',
'st_1yr_tchr_trnovr_pct',
'lea_tchyrs_0thru3_pct',
'Category_Cd_T',
]]

# Dataset Dimension with picked variables
schoolData_pick.shape

(9731, 50)

### Training/Testing Splitting

To prepare our official SVM and Logistic regression modeling procedure, we need to split the dataset into training and testing set. We use the ShuffleSplit criteria below. We use a proportion of 80/20 for the training/testing dataset.

In [42]:
# Using ShuffleSplit for Training/Testing Split 
from sklearn.model_selection import ShuffleSplit

num_cv_iterations = 3
num_instances = len(schoolData_SPG)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [51]:
# Split X and Y into Training and Testing dataset
for train_indices, test_indices in cv_object.split(schoolData_pick,schoolData_SPG): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = schoolData_pick.values[train_indices]
    y_train = schoolData_SPG.values[train_indices]
    Xall_train = schoolData.values[train_indices]
    
    X_test = schoolData_pick.values[test_indices]
    y_test = schoolData_SPG.values[test_indices]
    Xall_test = schoolData.values[test_indices]

 With our feature selection (51 variables), check and make sure it contains most information. The way we conduct the validation is by running another random forest and check the accuracy. This accuracy of the current model (acc=0.733) is close to the model utilizing all the variables (acc=0.736). So we varify that our selection is reasonable.

In [53]:
# Random Forest fitting with selective variables
clf_rf=RandomForestClassifier(n_estimators=100)
clf_rf.fit(X_train,y_train)
y_pred_rf=clf_rf.predict(X_test)

# Random Forest fitting with all variables
clf_rf_all=RandomForestClassifier(n_estimators=100)
clf_rf_all.fit(Xall_train,y_train)
y_pred_rf_all=clf_rf_all.predict(Xall_test)

print("Model with Selective Variables Accuracy:",metrics.accuracy_score(y_test, y_pred_rf))
print("Model with All Variables Accuracy:",metrics.accuracy_score(y_test, y_pred_rf_all))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Model with Selective Variables Accuracy: 0.733949666153056
Model with All Variables Accuracy: 0.7360041088854649


### Solving Data Imbalance

We find that our response variable (SPG_Grade) has a data balancing problem: The grade C is dominate the dataset with the largest propertion, followed by B or D. A and F are just a small propertion of the whole dataset. To solve this problem, we are using the crieteria below.

#### Test 3 types of sampler: UnderSample all cases, UnderSample majority class, SMOTE Oversample

Below we conduct 3 seperate procedures to produce 3 seperate sets of X and y from the original dataset. Those sets will be evaluate against each other and we will decide the best data balancing strategy to use.

In [54]:
from imblearn.under_sampling import RandomUnderSampler

# Undersample the dataset using the RandomUnderSampler 
# Set all class sizes equal to the smallest class.
us = RandomUnderSampler()
X_us, y_us = us.fit_sample(schoolData_pick, schoolData_SPG)

# Plot the new class distributions for y using the same funnction as above. 
#plot_class_dist(y_us)

In [55]:
from imblearn.under_sampling import RandomUnderSampler

# Undersample the dataset using the RandomUnderSampler 
# Undersample the majority class only
us = RandomUnderSampler(sampling_strategy='majority')
X_us_maj, y_us_maj = us.fit_sample(schoolData_pick, schoolData_SPG)

# Plot the new class distributions for y using the same funnction as above. 
#plot_class_dist(y_us_maj)

In [56]:
from imblearn.over_sampling import SMOTE

# Oversample using SMOTE
# Make all classes the same size as the majority class.
# WARNING - This can exponentially increase the size of the input dataset.
sm = SMOTE(sampling_strategy='not majority')
X_sm, y_sm = sm.fit_sample(schoolData_pick, schoolData_SPG)

# Plot the new class distributions for y using the same funnction as above. 
#plot_class_dist(y_sm)

By comparing the results for the 3 different methods dealing with data imbalance, we run random forest with the datasets we generated. Accuracy are measured with the results of the variables. As shown the SMOTE strategy get the highest accuracy. It will be used for the later SVM and Logistic regression fitting.

In [58]:
#Training using Undersample Method1 
for train_indices, test_indices in cv_object.split(X_us,y_us): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X_us.values[train_indices]
    y_train = y_us.values[train_indices]
    
    X_test = X_us.values[test_indices]
    y_test = y_us.values[test_indices]
    

clfus=RandomForestClassifier(n_estimators=100)
clfus.fit(X_train,y_train)
y_pred_us=clfus.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred_us))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Accuracy: 0.7586206896551724


In [59]:
#Training using Undersample Method2
for train_indices, test_indices in cv_object.split(X_us_maj,y_us_maj): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X_us_maj.values[train_indices]
    y_train = y_us_maj.values[train_indices]
    
    X_test = X_us_maj.values[test_indices]
    y_test = y_us_maj.values[test_indices]
    
clfusm=RandomForestClassifier(n_estimators=100)
clfusm.fit(X_train,y_train)
y_pred_usm=clfusm.predict(X_test)
    
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_usm))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Accuracy: 0.8526672311600338


In [60]:
#Training using Oversample Method
for train_indices, test_indices in cv_object.split(X_sm,y_sm): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X_sm.values[train_indices]
    y_train = y_sm.values[train_indices]
    
    X_test = X_sm.values[test_indices]
    y_test = y_sm.values[test_indices]

clfsm=RandomForestClassifier(n_estimators=100)
clfsm.fit(X_train,y_train)
y_pred_sm=clfsm.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred_sm))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Accuracy: 0.9219757747365109


## Model Creation
### SVM Work


### Logistic Regression

## Input Feature Importance

## Support Vectors Interpretation

## Model Advantages